In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
from sklearn import linear_model

In [2]:
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

**Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.**

In [3]:
houseprices.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', 'firstflrsf', 'secondflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paved

In [51]:
houseprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
id               1460 non-null int64
mssubclass       1460 non-null int64
mszoning         1460 non-null object
lotfrontage      1201 non-null float64
lotarea          1460 non-null int64
street           1460 non-null object
alley            91 non-null object
lotshape         1460 non-null object
landcontour      1460 non-null object
utilities        1460 non-null object
lotconfig        1460 non-null object
landslope        1460 non-null object
neighborhood     1460 non-null object
condition1       1460 non-null object
condition2       1460 non-null object
bldgtype         1460 non-null object
housestyle       1460 non-null object
overallqual      1460 non-null int64
overallcond      1460 non-null int64
yearbuilt        1460 non-null int64
yearremodadd     1460 non-null int64
roofstyle        1460 non-null object
roofmatl         1460 non-null object
exterior1st      1460 non-n

In [4]:
null_counts = houseprices.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

poolqc          1453
miscfeature     1406
alley           1369
fence           1179
fireplacequ      690
lotfrontage      259
garageyrblt       81
garagetype        81
garagefinish      81
garagequal        81
garagecond        81
bsmtfintype2      38
bsmtexposure      38
bsmtfintype1      37
bsmtcond          37
bsmtqual          37
masvnrarea         8
masvnrtype         8
electrical         1
dtype: int64

In [5]:
null_col = ["poolqc","miscfeature","alley", "fence", "fireplacequ","lotfrontage", "garageyrblt","garagetype","garagefinish",      
"garagequal","garagecond","bsmtfintype2","bsmtexposure", "bsmtfintype1","bsmtcond", "bsmtqual", "masvnrarea","masvnrtype",       
"electrical"]  

houseprices[null_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 19 columns):
poolqc          7 non-null object
miscfeature     54 non-null object
alley           91 non-null object
fence           281 non-null object
fireplacequ     770 non-null object
lotfrontage     1201 non-null float64
garageyrblt     1379 non-null float64
garagetype      1379 non-null object
garagefinish    1379 non-null object
garagequal      1379 non-null object
garagecond      1379 non-null object
bsmtfintype2    1422 non-null object
bsmtexposure    1422 non-null object
bsmtfintype1    1423 non-null object
bsmtcond        1423 non-null object
bsmtqual        1423 non-null object
masvnrarea      1452 non-null float64
masvnrtype      1452 non-null object
electrical      1459 non-null object
dtypes: float64(3), object(16)
memory usage: 216.8+ KB


In [6]:
houseprices[houseprices["garagequal"].isnull()].loc[:,["garagequal","alley", "fireplacequ", "fence", "miscfeature","lotfrontage", "garageyrblt"]]

,garagequal,alley,fireplacequ,fence,miscfeature,lotfrontage,garageyrblt
39,None,None,None,None,None,65.0,NaN
48,None,None,None,None,None,33.0,NaN
78,None,None,None,None,None,72.0,NaN
88,None,None,None,MnPrv,None,105.0,NaN
89,None,None,None,None,None,60.0,NaN
99,None,None,None,None,Shed,77.0,NaN
100,None,Grvl,None,None,None,60.0,NaN
109,None,None,None,None,None,85.0,NaN
126,None,None,None,None,None,60.0,NaN
128,None,None,TA,None,None,55.0,NaN


In [7]:
houseprices = houseprices.drop(columns = ["lotfrontage","garageyrblt", "garagetype", "garagefinish", "garagequal","garagecond","bsmtfintype2","bsmtexposure", "bsmtfintype1", "bsmtcond", "bsmtqual", "masvnrarea",    
"masvnrtype","electrical"])

In [8]:
print(houseprices["fence"].value_counts(dropna = False))
print(houseprices["alley"].value_counts(dropna = False))
print(houseprices["poolqc"].value_counts(dropna = False))
print(houseprices["fireplacequ"].value_counts(dropna = False))
print(houseprices["miscfeature"].value_counts(dropna = False))

houseprices = houseprices.drop(columns = ["fence","alley","poolqc","fireplacequ","miscfeature"])

NaN      1179
MnPrv     157
GdPrv      59
GdWo       54
MnWw       11
Name: fence, dtype: int64
NaN     1369
Grvl      50
Pave      41
Name: alley, dtype: int64
NaN    1453
Gd        3
Ex        2
Fa        2
Name: poolqc, dtype: int64
NaN    690
Gd     380
TA     313
Fa      33
Ex      24
Po      20
Name: fireplacequ, dtype: int64
NaN     1406
Shed      49
Gar2       2
Othr       2
TenC       1
Name: miscfeature, dtype: int64


In [9]:
houseprices.columns

Index(['id', 'mssubclass', 'mszoning', 'lotarea', 'street', 'lotshape',
       'landcontour', 'utilities', 'lotconfig', 'landslope', 'neighborhood',
       'condition1', 'condition2', 'bldgtype', 'housestyle', 'overallqual',
       'overallcond', 'yearbuilt', 'yearremodadd', 'roofstyle', 'roofmatl',
       'exterior1st', 'exterior2nd', 'exterqual', 'extercond', 'foundation',
       'bsmtfinsf1', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'firstflrsf', 'secondflrsf', 'lowqualfinsf',
       'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath',
       'bedroomabvgr', 'kitchenabvgr', 'kitchenqual', 'totrmsabvgrd',
       'functional', 'fireplaces', 'garagecars', 'garagearea', 'paveddrive',
       'wooddecksf', 'openporchsf', 'enclosedporch', 'threessnporch',
       'screenporch', 'poolarea', 'miscval', 'mosold', 'yrsold', 'saletype',
       'salecondition', 'saleprice'],
      dtype='object')

In [10]:
houseprices.isnull().sum()

id               0
mssubclass       0
mszoning         0
lotarea          0
street           0
lotshape         0
landcontour      0
utilities        0
lotconfig        0
landslope        0
neighborhood     0
condition1       0
condition2       0
bldgtype         0
housestyle       0
overallqual      0
overallcond      0
yearbuilt        0
yearremodadd     0
roofstyle        0
roofmatl         0
exterior1st      0
exterior2nd      0
exterqual        0
extercond        0
foundation       0
bsmtfinsf1       0
bsmtfinsf2       0
bsmtunfsf        0
totalbsmtsf      0
                ..
centralair       0
firstflrsf       0
secondflrsf      0
lowqualfinsf     0
grlivarea        0
bsmtfullbath     0
bsmthalfbath     0
fullbath         0
halfbath         0
bedroomabvgr     0
kitchenabvgr     0
kitchenqual      0
totrmsabvgrd     0
functional       0
fireplaces       0
garagecars       0
garagearea       0
paveddrive       0
wooddecksf       0
openporchsf      0
enclosedporch    0
threessnporc

**Now, split your data into train and test sets where 20% of the data resides in the test set.**

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
Y = houseprices['saleprice']

#print(categorical.select_dtypes(exclude=[object]).columns)

numerical_cols = [col_name for col_name in houseprices.select_dtypes(exclude=[object]).columns 
                  if col_name not in ["id", "saleprice"] ]

X = houseprices[numerical_cols]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

The number of observations in training set is 1168
The number of observations in test set is 292


**Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!**

In [12]:
#Lasso
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=0.1) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))

R-squared of the model on the training set is: 0.8149282336159746
-----Test set statistics-----
R-squared of the model on the test set is: 0.6830252422952159


In [13]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=0.1, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))

R-squared of the model on the training set is: 0.8142021201128833
-----Test set statistics-----
R-squared of the model on the test set is: 0.6787636550694531


In [14]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=0.1) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))

R-squared of the model on the training set is: 0.8149282279107182
-----Test set statistics-----
R-squared of the model on the test set is: 0.6830153793061078


In [33]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

X = houseprices[numerical_cols]
Y = houseprices['saleprice']

lasso = linear_model.Lasso(alpha = 1000)

print(cross_val_score(lasso, X, Y, cv=3))  

#changing alpha doesn't influence accuracy score for the folds to be more similar to each other. 

#gridsearch helps us to process all alpha values at once. return accuracy score and hyperparameter from the list of parameters we give. 
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {"alpha":[0.01,0.1,1,10,100,1000,1200,10000]}
lasso = linear_model.Lasso()
clf = GridSearchCV(lasso, parameters, cv=3)
clf.fit(X, Y)
clf.best_params_

[0.84432342 0.79014146 0.70335409]


{'alpha': 1000}

In [37]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

X = houseprices[numerical_cols]
Y = houseprices['saleprice']

ridge = linear_model.Ridge(alpha = 1000)

print(cross_val_score(ridge, X, Y, cv=3))  

#changing alpha doesn't influence accuracy score for the folds to be more similar to each other. 

#gridsearch helps us to process all alpha values at once. return accuracy score and hyperparameter from the list of parameters we give. 
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {"alpha":[0.01,0.1,1,10,100,1000,1200,10000]}
ridge = linear_model.Ridge()
clf = GridSearchCV(ridge, parameters, cv=3)
clf.fit(X, Y)
clf.best_params_

[0.81571323 0.78028084 0.67676815]


{'alpha': 100}

In [39]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

X = houseprices[numerical_cols]
Y = houseprices['saleprice']

ElasticNet = linear_model.ElasticNet(alpha = 1000)

print(cross_val_score(ElasticNet, X, Y, cv=3))  

#changing alpha doesn't influence accuracy score for the folds to be more similar to each other. 

#gridsearch helps us to process all alpha values at once. return accuracy score and hyperparameter from the list of parameters we give. 
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {"alpha":[0.01,0.1,1,10,100,1000,1200,10000]}
ElasticNet = linear_model.ElasticNet()
clf = GridSearchCV(ElasticNet, parameters, cv=3)
clf.fit(X, Y)
clf.best_params_

[0.76512916 0.73876179 0.60332608]


{'alpha': 0.1}

Among the three linear regression models, the differences between the accuracy scores per folds are similar. However, Lasso Model has the highest accuracy score so we go with Lasso. 